# Laboratorium nr 2

## Konfiguracja


Poniższe narzędzie zostało zmodyfikowane, tak aby można było z niego korzystać w nowszych IDE, które nie wspierają `%matplotlib notebook`.
Aby korzystać z `%matplotlib widget` należy doinstalować pakiet `ipympl` np. za pomocą polecenia `pip install ipympl`. Jeżeli po zainstalowaniu będą występować błędy, należy korzystać z backendu `notebook`.

Wykorzystywane biblioteki:
- matplotlib
- numpy
- ipympl

Nowe funkcje:
- Dodanie do konstruktora klasy Scene argumentu `title`, który pozwala na ustawienie tytułu danej sceny
- Dodanie do konstruktora klasy Plot argumentu `title`, który pozwala na ustawienie tytułu wykresu / okna widżetu


In [ ]:
# Wybór backendu matplotlib
# %matplotlib notebook
%matplotlib widget

# Narzędzie jest oparte o kilka zewnętrznych bibliotek, które potrzebujemy najpierw zaimportować.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

# Parametr określający jak blisko (w odsetku całego widocznego zakresu) punktu początkowego 
# wielokąta musimy kliknąć, aby go zamknąć.
TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

# Klasa ta trzyma obecny stan wykresu oraz posiada metody, które mają zostać wykonane
# po naciśnięciu przycisków.
class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []
        # ADDED: save scene title
        self.scene_title = self.scenes[0].title if len(self.scenes) and self.scenes[0].title else ""

    def set_axes(self, ax):
        self.ax = ax
        
    # Metoda ta obsługuje logikę przejścia do następnej sceny.
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        # ADDED: Change scene title
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = True)

    # Metoda ta obsługuje logikę powrotu do poprzedniej sceny.
    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        # ADDED: Change scene title
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = True)
        
    # Metoda ta aktywuje funkcję rysowania punktów wyłączając równocześnie rysowanie 
    # odcinków i wielokątów.
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))
            
    # Metoda ta aktywuje funkcję rysowania odcinków wyłączając równocześnie
    # rysowanie punktów i wielokątów.     
    def add_line(self, event):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    # Metoda ta aktywuje funkcję rysowania wielokątów wyłączając równocześnie
    # rysowanie punktów i odcinków.
    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []
        
    # Metoda odpowiedzialna za właściwą logikę rysowania nowych elementów. W
    # zależności od włączonego trybu dodaje nowe punkty, początek, koniec odcinka
    # lub poszczególne wierzchołki wielokąta. Istnieje ciekawa logika sprawdzania
    # czy dany punkt jest domykający dla danego wielokąta. Polega ona na tym, że
    # sprawdzamy czy odległość nowego punktu od początkowego jest większa od
    # średniej długości zakresu pomnożonej razy parametr TOLERANCE.   
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:    
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)
    
    # Metoda odpowiedzialna za narysowanie całego wykresu. Warto zauważyć,
    # że zaczyna się ona od wyczyszczenia jego wcześniejszego stanu. Istnieje w
    # niej nietrywialna logika zarządzania zakresem wykresu, tak żeby, w zależności
    # od ustawionego parametru autoscaling, uniknąć sytuacji, kiedy dodawanie
    # nowych punktów przy brzegu obecnie widzianego zakresu powoduje niekorzystne
    # przeskalowanie.
    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
            
        # ADDED: Add scene title
        fig = plt.gcf() # Get current window
        fig.suptitle(self.scene_title)
        
        plt.draw()


### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [ ]:
# Klasa Scene odpowiada za przechowywanie elementów, które mają być
# wyświetlane równocześnie. Konkretnie jest to lista PointsCollection i
# LinesCollection.
class Scene:
    def __init__(self, points=[], lines=[], title=""):
        self.points=points
        self.lines=lines
        # ADDED: Scene title
        self.title = '\n'+title if title != "" else ""

# Klasa PointsCollection gromadzi w sobie punkty jednego typu, a więc takie,
# które zostaną narysowane w takim samym kolorze i stylu. W konstruktorze
# przyjmuje listę punktów rozumianych jako pary współrzędnych (x, y). Parametr
# kwargs jest przekazywany do wywołania funkcji z biblioteki MatPlotLib przez
# co użytkownik może podawać wszystkie parametry tam zaproponowane.        
class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points = self.points + points

# Klasa LinesCollection podobnie jak jej punktowy odpowiednik gromadzi
# odcinki tego samego typu. Tworząc ją należy podać listę linii, gdzie każda
# z nich jest dwuelementową listą punktów – par (x, y). Parametr kwargs jest
# przekazywany do wywołania funkcji z biblioteki MatPlotLib przez co użytkownik
# może podawać wszystkie parametry tam zaproponowane.
class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

# Klasa Plot jest najważniejszą klasą w całym programie, ponieważ agreguje
# wszystkie przygotowane sceny, odpowiada za stworzenie wykresu i przechowuje
# referencje na przyciski, dzięki czemu nie będą one skasowane podczas tzw.
# garbage collectingu.
class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None, title=""):
        # ADDED: plot title
        self.plot_title = title
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
    
    # Ta metoda ma szczególne znaczenie, ponieważ konfiguruje przyciski i
    # wykonuje tym samym dość skomplikowaną logikę. Zauważmy, że konfigurując każdy
    # przycisk podajemy referencję na metodę obiektu _Button_callback, która
    # zostanie wykonana w momencie naciśnięcia.
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    # Metoda toJson() odpowiada za zapisanie stanu obiektu do ciągu znaków w
    # formacie JSON.
    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])    
    
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None
        
    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None
    
    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None
    
    # Główna metoda inicjalizująca wyświetlanie wykresu.
    def draw(self):
        plt.close()
        # ADDED: plot title
        fig = plt.figure(num=self.plot_title)
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()
        

### Przykłady użycia

##### Proste rysowanie

Rysowanie prostego rysunku złożonego ze statycznych punktów i odcinków jest stosunkowo proste. Należy utworzyć zmienną z obiektem `Plot`, który przyjmuje w konstruktorze listę obiektów `PointsCollection` jako parametr `points` oraz listę `LinesCollection` jako parametr `lines`. Następnie należy wywołać jego metodę `draw()`.

```python
plot = Plot(points=[PointsCollection([(1, 2), (3, 1.5), (2, -1)]),
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")],
            lines=[LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])])
plot.draw()     
```

##### Rysunek dynamiczny

To narzędzie pozwala na wizualizację "rysunków dynamicznych", a więc takich, które składają się z wielu różnych obrazów. Może to być przydatne np. podczas wizualizacji kolejnych kroków danego algorytmu. Poszczególne klatki określane są jako obiekty `Scene`, które przyjmują listę `PointsCollection` oraz listę `LinesCollection` w swoim konstruktorze. Obiekt `Plot` przyjmuje w parametrze konstruktora `scenes` właśnie listę takich scen.

Na rysunku przyciski `Następny` i `Poprzedni` służą do nawigacji pomiędzy nimi.

```python
scenes=[Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)]), 
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])], "Scene 1"), 
        Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)], color='red'), 
               PointsCollection([(5, -2), (2, 2), (-2, 1)], color='black')], 
              [LinesCollection([[(-1,2),(-2,3)], [(0,-1),(-1,0)]])],"Scene 2")]

plot=Plot(scenes = scenes)
plot.draw()
```

##### Zapis i odczyt z pliku

Klasa Plot posiada metodę `toJson()`, która zwraca string zawierający listę scen w formacie JSON. Taki string można normalnie zapisać do pliku stosując normalne sposoby dostępne w Pythonie. Wczytanie listy scen z pliku dokonuje się poprzez podanie parametru `json` w kostruktorze `Plot`.

```python
scenes=[Scene([PointsCollection([(-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])]), 
        Scene([PointsCollection([(1, 2), (3, 1.5)], color='red'), 
               PointsCollection([(5, -2)], color='black')])]
plot = Plot(scenes)

with open('somefile.json', 'w') as file:
    file.write(plot.toJson())
    
#somefile.json: [{"points": [[[-2, -1]]], "lines": [[[[1, 2], [2, 3]], [[0, 1], [1, 0]]]]}, {"points": [[[1.0, 2.0], [3.0, 1.5]], [[5, -2]]], "lines": []}]
    
with open('somefile.json', 'r') as file:
    json = file.read()
    
plot2 = Plot(json=json)
plot2.draw()
```

##### Operowanie na dodanych punktach

Możemy bardzo łatwo rysować nowe punkty i linie. Aby je później wykorzystać sugerowanym sposobem jest utworzenie nowej komórki, w której utworzymy nowy `Plot` i skorzystamy z metod `get_added_points()` i `get_added_points()` lub `get_added_elements()` (zwraca scenę) z pierwszego `Plot`u.

```python 
#Dodaj tu punkty, figury i/lub odcinki!
plot1 = Plot(points=[PointsCollection([(-2,-1),(5,3)], color='red')])
plot1.draw()

plot2 = Plot([plot1.get_added_elements()])
plot2.draw()
```

## Generacja zbiorów punktów

### Zbiór danych A
Zbiór zawierający 100 losowo wygenerowanych punktów o współrzędnych z przedziału $[-100, 100]$

In [ ]:
def generate_dataset_a(DATASET_A_RANGE=(-100,100),DATASET_A_SIZE=100):
    dataset_a = np.random.uniform(DATASET_A_RANGE[0],DATASET_A_RANGE[1], size=(DATASET_A_SIZE,2))
    return PointsCollection(dataset_a)

default_dataset_a = generate_dataset_a()
plot = Plot(points=[default_dataset_a], title="Zbiór danych A")
plot.draw()

### Zbiór danych B
Zbiór zawierający 100 losowo wygenerowanych punktów leżących na okręgu o środku $(0,0)$ i promieniu $R=10$

In [ ]:
def generate_dataset_b(DATASET_B_CIRCLE_CENTER=(0,0),DATASET_B_SIZE=100,DATASET_B_CIRCLE_RADIUS=10):
    angle = np.random.uniform(0,2*np.pi, size=DATASET_B_SIZE)

    dataset_b = []
    for i in range(DATASET_B_SIZE):
        dataset_b.append((DATASET_B_CIRCLE_CENTER[0] + DATASET_B_CIRCLE_RADIUS * np.sin(angle[i]),DATASET_B_CIRCLE_CENTER[1] + DATASET_B_CIRCLE_RADIUS * np.cos(angle[i])))

    return PointsCollection(dataset_b)
    
default_dataset_b = generate_dataset_b()
plot = Plot(points=[default_dataset_b], title="Zbiór danych B")
plot.draw()

### Zbiór danych C
Zbiór zawierający $100$ losowo wygenerowanych punktów leżących na bokach prostokąta o wierzchołkach $(-10, 10)$, $(-10,-10)$, $(10,-10)$, $(10,10)$

In [ ]:
def generate_dataset_c(DATASET_C_RECTANGLE_VERTEX= [(-10,-10), (10,-10),(10,10),(-10,10)] ,DATASET_C_SIZE=100):
    # DATASET_C_RECTANGLE_VERTEX - sorted anticlockwise and starting from bottom left list of points

    # Let's create a segment in range (0, sum(sides)). Then we divide this segment accordingly to side lengths.

    first_side = DATASET_C_RECTANGLE_VERTEX[1][0] - DATASET_C_RECTANGLE_VERTEX[0][0]
    second_side = DATASET_C_RECTANGLE_VERTEX[2][1] - DATASET_C_RECTANGLE_VERTEX[1][1]

    # Format: [a, b]
    rectangle_sides = [[0,first_side]]
    rectangle_sides.append([rectangle_sides[-1][1], rectangle_sides[-1][1] + second_side])
    rectangle_sides.append([rectangle_sides[-1][1], rectangle_sides[-1][1] + first_side])
    rectangle_sides.append([rectangle_sides[-1][1], rectangle_sides[-1][1] + second_side])

    points = np.random.uniform(rectangle_sides[0][0], rectangle_sides[-1][1], size=DATASET_C_SIZE)

    dataset_c = []

    for point in points:
        for i in range(4):
            if rectangle_sides[i][0] <= point < rectangle_sides[i][1]:
                if i==0:
                    substract = rectangle_sides[i][0] - DATASET_C_RECTANGLE_VERTEX[i][0]
                    dataset_c.append((point - substract, DATASET_C_RECTANGLE_VERTEX[i][1]))
                elif i==1:
                    substract = rectangle_sides[i][0] - DATASET_C_RECTANGLE_VERTEX[i][1]
                    dataset_c.append((DATASET_C_RECTANGLE_VERTEX[i][0],point - substract))
                elif i==2:
                    substract = rectangle_sides[i][0] - DATASET_C_RECTANGLE_VERTEX[i+1][0]
                    dataset_c.append((point - substract, DATASET_C_RECTANGLE_VERTEX[i][1]))       
                else:
                    substract = rectangle_sides[i][0] - DATASET_C_RECTANGLE_VERTEX[0][1]
                    dataset_c.append((DATASET_C_RECTANGLE_VERTEX[i][0],point - substract))

    return PointsCollection(dataset_c)
    
default_dataset_c = generate_dataset_c()
plot = Plot(points=[default_dataset_c], title="Zbiór danych C")
plot.draw()

### Zbiór danych D
Zbiór zawierający  wierzchołki  kwadratu  $(0,  0)$,  $(10,  0)$,  $(10,  10)$,  $(0,  10)$  oraz  punkty  wygenerowane  losowo  w  sposób  następujący:  po  $25$  punktów  na  dwóch  bokach kwadratu leżących na osiach i po $20$ punktów na przekątnych kwadratu. 

In [ ]:
def generate_dataset_d(DATASET_D_NO_OF_SIDE_POINTS=25,DATASET_D_NO_OF_DIAGONAL_POINTS=20,DATASET_D_SQUARE_VERTEX = [(0,0), (10,0),(10,10),(0,10)]):
    # DATASET_D_SQUARE_VERTEX - sorted anticlockwise and starting from bottom left

    # Create random points for sides
    s1, s2 = [], []
    for _ in range(DATASET_D_NO_OF_SIDE_POINTS):
        x1 = np.random.uniform(DATASET_D_SQUARE_VERTEX[0][0],DATASET_D_SQUARE_VERTEX[1][0])
        x2 = np.random.uniform(DATASET_D_SQUARE_VERTEX[0][1],DATASET_D_SQUARE_VERTEX[3][1])
        s1.append((x1,DATASET_D_SQUARE_VERTEX[0][1]))
        s2.append((DATASET_D_SQUARE_VERTEX[0][0],x2))

    d1,d2 = [], []

    # Create random points for diagonals
    for _ in range(DATASET_D_NO_OF_DIAGONAL_POINTS):
        x1 = np.random.uniform(DATASET_D_SQUARE_VERTEX[0][0],DATASET_D_SQUARE_VERTEX[2][0])
        x2 = np.random.uniform(DATASET_D_SQUARE_VERTEX[0][0],DATASET_D_SQUARE_VERTEX[2][0])
        d1.append((x1,x1))
        d2.append((x2,-x2+DATASET_D_SQUARE_VERTEX[3][1]))

    dataset_d = DATASET_D_SQUARE_VERTEX.copy()
    dataset_d.extend(s1)
    dataset_d.extend(s2)
    dataset_d.extend(d1)
    dataset_d.extend(d2)

    return PointsCollection(dataset_d)
    
default_dataset_d = generate_dataset_d()
plot = Plot(points=[default_dataset_d], title="Zbiór danych D")
plot.draw()

## Implementacje algorytmów do wyznaczania otoczki wypukłej

### Funkcje pomocnicze oraz dodatkowe biblioteki

In [ ]:
from functools import cmp_to_key
from collections import deque
from time import time, process_time

def det3x3(a,b,c):
    matrix = [[],[],[]]
    matrix[0] = [a[0], a[1], 1]
    matrix[1] = [b[0], b[1], 1]
    matrix[2] = [c[0], c[1], 1]

    f1 = matrix[0][0] * matrix[1][1] * matrix[2][2]
    f2 = matrix[0][1] * matrix[1][2] * matrix[2][0]
    f3 = matrix[1][0] * matrix[2][1] * matrix[0][2]
    f4 = matrix[0][2] * matrix[1][1] * matrix[2][0]
    f5 = matrix[1][2] * matrix[2][1] * matrix[0][0]
    f6 = matrix[1][0] * matrix[0][1] * matrix[2][2]
    return f1+f2+f3-f4-f5-f6

def distance(p1,p2):
    # Using taxicab metric (because we check points on the same ray)
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

### Algorytm Grahama

Opis algorytmu

In [ ]:
def graham_algorithm(points: PointsCollection, scenes=True, filename="",calculate_exec_time=True):
    # Algorithm configuration
    epsilon = 1e-10
    start_time = process_time()

    # ================ GRAHAM ALGORITHM ================

    # Find bottom-left point
    raw_points = points.points

    min_point = (np.inf, np.inf)
    for a,b in raw_points:
        if b < min_point[1]:
            min_point = (a,b)
        elif b == min_point[1] and a < min_point[0]:
            min_point = (a,b)


    # Create new list (without min_point)
    considered_points = []
    for a,b in raw_points:
        if a == min_point[0] and b == min_point[1]:
            continue
        considered_points.append((a,b))

    # Sort points
    def compare_fn(a,b):
        nonlocal min_point

        # if a > b then determinant is > 0
        determinant = det3x3(a, b, min_point)
        if -epsilon<=determinant<=epsilon:
            # Compare by distance from min_point
            d1 = distance(min_point,a)
            d2 = distance(min_point,b)

            if d1 < d2:
                return -1
            elif d1 > d2:
                return 1
            
            return 0
        elif determinant > epsilon:
            return -1
        else:
            return 1

    considered_points = sorted(considered_points,key=cmp_to_key(compare_fn))
    considered_points.insert(0,min_point)

    # First convex-hull points
    s = deque()
    selected_points = []

    s.append(considered_points[0])
    selected_points.append(considered_points[0])

    # ================ GRAHAM ALGORITHM MODIFICATION ================
    # Remove s[1] if nessecary (when 3 points are collinear)
    if not (-epsilon <= det3x3(considered_points[0],considered_points[2],considered_points[1]) <= epsilon):
        s.append(considered_points[1])
        selected_points.append(considered_points[1])
    # ================ END OF MODIFICATION ================

    s.append(considered_points[2])
    selected_points.append(considered_points[2])

    i = 3
    
    # Scene configuration
    scene_container = []
    considered_lines = [(considered_points[0], considered_points[1]), (considered_points[1], considered_points[2])]
    if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green")], title="Początek algorytmu Grahama"))

    while i<len(considered_points):
        determinant = det3x3(s[-2],s[-1],considered_points[i])
        if determinant > epsilon:
            s.append(considered_points[i])
            selected_points.append(s[-1])
            i+=1

            # Add scene
            considered_lines.append((s[-2], s[-1]))
            if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green")], title="Szukanie potencjalnych punktów otoczki"))

        # ================ GRAHAM ALGORITHM MODIFICATION ================
        elif -epsilon <= determinant <= epsilon:
            s.pop()
            selected_points.pop()
            considered_lines.pop()

            s.append(considered_points[i])
            selected_points.append(s[-1])
            i+=1

            # Add scene
            considered_lines.append((s[-2], s[-1]))
            if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green")], title="Szukanie potencjalnych punktów otoczki"))
        # ================ END OF MODIFICATION ================

        else:
            s.pop()

            # Configure scene utils
            selected_points.pop()
            considered_lines.pop()
    

    # Configure scene utils
    selected_points.append(s[0])
    considered_lines.append((s[-1], s[0]))
    
    end_time = process_time()

    # ================ END OF ALGORITHM ================
    
    scene_container.insert(0,Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="red")], title="Rezultat algorytmu Grahama"))

    # Save to file
    if filename != "":
        selected_points.pop()
        test_plot = Plot(points=[PointsCollection(selected_points)])
        with open(f"{filename}.json", 'w') as file:
            file.write(test_plot.toJson())

    if calculate_exec_time:
        print(f"Czas działania programu: {end_time - start_time}s.")

    return considered_lines.copy(),selected_points.copy(), scene_container


def pure_graham_algorithm(points: PointsCollection):
    # Algorithm without additional features for measuring time
    points = PointsCollection(points.points[:])

    # Algorithm configuration
    epsilon = 1e-10
    start_time = process_time()

    # ================ GRAHAM ALGORITHM ================

    # Find bottom-left point
    raw_points = points.points

    min_point = (np.inf, np.inf)
    for a,b in raw_points:
        if b < min_point[1]:
            min_point = (a,b)
        elif b == min_point[1] and a < min_point[0]:
            min_point = (a,b)


    # Create new list (without min_point)
    considered_points = []
    for a,b in raw_points:
        if a == min_point[0] and b == min_point[1]:
            continue
        considered_points.append((a,b))

    # Sort points
    def compare_fn(a,b):
        nonlocal min_point

        # if a > b then determinant is > 0
        determinant = det3x3(a, b, min_point)
        if -epsilon<=determinant<=epsilon:
            # Compare by distance from min_point
            d1 = distance(min_point,a)
            d2 = distance(min_point,b)

            if d1 < d2:
                return -1
            elif d1 > d2:
                return 1
            
            return 0
        elif determinant > epsilon:
            return -1
        else:
            return 1

    considered_points = sorted(considered_points,key=cmp_to_key(compare_fn))
    considered_points.insert(0,min_point)

    # First convex-hull points
    s = deque()
    s.append(considered_points[0])

    # ================ GRAHAM ALGORITHM MODIFICATION ================
    # Remove s[1] if nessecary (when 3 points are collinear)
    if not (-epsilon <= det3x3(considered_points[0],considered_points[2],considered_points[1]) <= epsilon):
        s.append(considered_points[1])
    # ================ END OF MODIFICATION ================

    s.append(considered_points[2])

    i = 3
    
    while i<len(considered_points):
        determinant = det3x3(s[-2],s[-1],considered_points[i])
        if determinant > epsilon:
            s.append(considered_points[i])
            i+=1

        # ================ GRAHAM ALGORITHM MODIFICATION ================
        elif -epsilon <= determinant <= epsilon:
            s.pop()
            s.append(considered_points[i])
            i+=1
        # ================ END OF MODIFICATION ================

        else:
            s.pop()
    
    
    end_time = process_time()

    # ================ END OF ALGORITHM ================
    

    print(f"Czas działania algorytmu: {end_time-start_time}s.")


### Algorytm Jarvisa

Opis algorytmu

In [ ]:
def jarvis_algorithm(points: PointsCollection, scenes=True, filename="", calculate_exec_time=True):
    # Algorithm configuration
    epsilon = 1e-10
    start_time = process_time()

    # ================ JARVIS ALGORITHM ================

    # Find bottom-left point
    raw_points = points.points

    min_point = (np.inf, np.inf)
    for a,b in raw_points:
        if b < min_point[1]:
            min_point = (a,b)
        elif b == min_point[1] and a < min_point[0]:
            min_point = (a,b)

    p0 = min_point
    p1 = raw_points[0] if raw_points[0][0] != min_point[0] and raw_points[0][1] != min_point[1] else raw_points[1]

    # Scene configuration
    scene_container = []
    selected_points = []
    added_lines = []
    considered_lines = []

    selected_points.append(p0)
    selected_points.append(p1)
    considered_lines.append((p0,p1))
    if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green"),LinesCollection(added_lines.copy(), color="red")], title="Początek algorytmu Jarvisa"))


    # First while execution - find p1    
    for point in raw_points:
        if point[0] == p0[0] and point[1] == p0[1]:
            continue

        # If point is on the right side of p0-p1 segment, use that point
        # Else if point is on a segment, check distances
        if det3x3(p0,p1,point) < -epsilon or (-epsilon<=det3x3(p0,p1,point) <= epsilon and distance(p0,point) > distance(p0, p1)):
            p1 = point

            # Add scene
            selected_points.pop()
            selected_points.append(p1)
            considered_lines.pop()
            considered_lines.append((p0,p1))
            if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green"),LinesCollection(added_lines.copy(), color="red")], title="Wyszukiwanie punktu z najmniejszym kątem"))

    # Add selected point p1 to plot
    added_lines.append((p0,p1))
    selected_points.append(p1)
    if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green"),LinesCollection(added_lines.copy(), color="red")], title="Wybrano punkt otoczki"))

    points_list = [p0,p1]
    p0 = p1

    while p0[0] != min_point[0] or p0[1] != min_point[1]:
        # Find p1    
        for point in raw_points:
            if point[0] == p0[0] and point[1] == p0[1]:
                continue
            
            # If point is on the right side of p0-p1 segment, use that point
            # Else if point is on a segment, check distances
            if det3x3(p0,p1,point) < -epsilon or (-epsilon<=det3x3(p0,p1,point) <= epsilon and distance(p0,point) > distance(p0, p1)):
                p1 = point

                # Add scene
                selected_points.pop()
                selected_points.append(p1)
                considered_lines.pop()
                considered_lines.append((p0,p1))
                if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green"),LinesCollection(added_lines.copy(), color="red")], title="Wyszukiwanie punktu z najmniejszym kątem"))

        # Add selected point p1 to plot
        added_lines.append((p0,p1))
        selected_points.append(p1)
        if scenes: scene_container.append(Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green"),LinesCollection(added_lines.copy(), color="red")], title="Wybrano punkt otoczki"))


        points_list.append(p1)
        p0 = p1

    end_time = process_time()

    # ================ END OF ALGORITHM ================  

    scene_container.insert(0,Scene(points=[points,PointsCollection(selected_points.copy(), color="orange")], lines=[LinesCollection(considered_lines.copy(), color="green"),LinesCollection(added_lines.copy(), color="red")], title="Rezultat algorytmu Jarvisa"))

    lines = []
    selected_points = [(points_list[0][0], points_list[0][1])]
    for i in range(1,len(points_list)):
        lines.append([(points_list[i-1][0], points_list[i-1][1]),(points_list[i][0], points_list[i][1])])
        selected_points.append((points_list[i][0], points_list[i][1]))

    lines.append([(points_list[-1][0], points_list[-1][1]),(points_list[0][0], points_list[0][1])])

    # Save to file
    if filename != "":
        selected_points.pop()
        test_plot = Plot(points=[PointsCollection(selected_points)])
        with open(f"{filename}.json", 'w') as file:
            file.write(test_plot.toJson())

    if calculate_exec_time:
        print(f"Czas działania programu: {end_time - start_time}s.")

    return lines, selected_points, scene_container



def pure_jarvis_algorithm(points: PointsCollection):
    # Algorithm configuration
    epsilon = 1e-10
    points = PointsCollection(points.points[:])
    start_time = process_time()

    # ================ JARVIS ALGORITHM ================

    # Find bottom-left point
    raw_points = points.points

    min_point = (np.inf, np.inf)
    for a,b in raw_points:
        if b < min_point[1]:
            min_point = (a,b)
        elif b == min_point[1] and a < min_point[0]:
            min_point = (a,b)

    p0 = min_point
    p1 = raw_points[0] if raw_points[0][0] != min_point[0] and raw_points[0][1] != min_point[1] else raw_points[1]

    # First while execution - find p1    
    for point in raw_points:
        if point[0] == p0[0] and point[1] == p0[1]:
            continue

        # If point is on the right side of p0-p1 segment, use that point
        # Else if point is on a segment, check distances
        if det3x3(p0,p1,point) < -epsilon or (-epsilon<=det3x3(p0,p1,point) <= epsilon and distance(p0,point) > distance(p0, p1)):
            p1 = point

    # points_list = [p0,p1]
    p0 = p1

    while p0[0] != min_point[0] or p0[1] != min_point[1]:
        # Find p1    
        for point in raw_points:
            if point[0] == p0[0] and point[1] == p0[1]:
                continue
            
            # If point is on the right side of p0-p1 segment, use that point
            # Else if point is on a segment, check distances
            if det3x3(p0,p1,point) < -epsilon or (-epsilon<=det3x3(p0,p1,point) <= epsilon and distance(p0,point) > distance(p0, p1)):
                p1 = point


        # points_list.append(p1)
        p0 = p1

    end_time = process_time()

    # ================ END OF ALGORITHM ================  

    print(f"Czas działania algorytmu: {end_time-start_time}s.")


## Test algorytmów dla badanych zestawów danych


### Zestaw danych A

#### Zestaw danych A.1
- liczba punktów: $100$
- przedziały dla współrzędnych: $[-100, 100]$

In [ ]:
pure_graham_algorithm(default_dataset_a)

In [ ]:
lines, points, scene_container = graham_algorithm(default_dataset_a, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.1")
plot.draw()

In [ ]:
pure_jarvis_algorithm(default_dataset_a)

In [ ]:
lines, points, scene_container = jarvis_algorithm(default_dataset_a, True)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.1")
plot.draw()

#### Zestaw danych A.2
- liczba punktów: $1000$
- przedziały dla współrzędnych: $[-300, 300]$

In [ ]:
dataset_a2 = generate_dataset_a((-300,300),1000)

In [ ]:
pure_graham_algorithm(dataset_a2)

In [ ]:
lines, points, scene_container = graham_algorithm(dataset_a2, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.2")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_a2)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_a2, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.2")
plot.draw()

#### Zestaw danych A.3
- liczba punktów: $2500$
- przedziały dla współrzędnych: $[-300, 300]$

In [ ]:
dataset_a3 = generate_dataset_a((-300,300),2500)

In [ ]:
pure_graham_algorithm(dataset_a3)

In [ ]:
lines, points, scene_container = graham_algorithm(dataset_a3, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.3")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_a3)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_a3, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.3")
plot.draw()

#### Zestaw danych A.4
- liczba punktów: $5000$
- przedziały dla współrzędnych: $[-300, 300]$

In [ ]:
dataset_a4 = generate_dataset_a((-300,300),5000)

In [ ]:
pure_graham_algorithm(dataset_a4)

In [ ]:
lines, points, scene_container = graham_algorithm(dataset_a4, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.4")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_a4)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_a4, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.4")
plot.draw()

#### Zestaw danych A.5
- liczba punktów: $10000$
- przedziały dla współrzędnych: $[-300, 300]$

In [ ]:
dataset_a5 = generate_dataset_a((-300,300),10000)

In [ ]:
pure_graham_algorithm(dataset_a5)

In [ ]:
lines, points, scene_container = graham_algorithm(dataset_a5, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.5")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_a5)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_a5, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.5")
plot.draw()

#### Zestaw danych A.6
- liczba punktów: $15000$
- przedziały dla współrzędnych: $[-300, 300]$

In [ ]:
dataset_a6 = generate_dataset_a((-300,300),15000)

In [ ]:
pure_graham_algorithm(dataset_a6)

In [ ]:
lines, points, scene_container = graham_algorithm(dataset_a6, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.6")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_a6)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_a6, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.6")
plot.draw()

#### Zestaw danych A.7
- liczba punktów: $25000$
- przedziały dla współrzędnych: $[-300, 300]$

In [ ]:
dataset_a7 = generate_dataset_a((-300,300),25000)


In [ ]:
pure_graham_algorithm(dataset_a7)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_a7, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.7")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_a7)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_a7, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.7")
plot.draw()

#### Zestaw danych A.8
- liczba punktów: $50000$
- przedziały dla współrzędnych: $[-300, 300]$

In [ ]:
dataset_a8 = generate_dataset_a((-300,300),50000)


In [ ]:
pure_graham_algorithm(dataset_a8)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_a8, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych A.8")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_a8)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_a8, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych A.8")
plot.draw()

### Zestaw danych B

#### Zestaw danych B.1
- liczba punktów: $100$
- środek okręgu: $(0,0)$
- promień okręgu: $R=100$

In [ ]:
pure_graham_algorithm(default_dataset_b)

In [ ]:
lines, points, scene_container = graham_algorithm(default_dataset_b, True)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.1")
plot.draw()

In [ ]:
pure_jarvis_algorithm(default_dataset_b)


In [ ]:
lines, points, scene_container = jarvis_algorithm(default_dataset_b, True)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.1")
plot.draw()

#### Zestaw danych B.2
- liczba punktów: $1000$
- środek okręgu: $(0,0)$
- promień okręgu: $R=300$

In [ ]:
dataset_b2 = generate_dataset_b((0,0),1000,300)

In [ ]:
pure_graham_algorithm(dataset_b2)

In [ ]:
lines, points, scene_container = graham_algorithm(dataset_b2, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.2")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_b2)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_b2, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.2")
plot.draw()

#### Zestaw danych B.3
- liczba punktów: $2500$
- środek okręgu: $(0,0)$
- promień okręgu: $R=300$

In [ ]:
dataset_b3 = generate_dataset_b((0,0),2500,300)


In [ ]:
pure_graham_algorithm(dataset_b3)

In [ ]:
lines, points, scene_container = graham_algorithm(dataset_b3, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.3")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_b3)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_b3, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.3")
plot.draw()

#### Zestaw danych B.4
- liczba punktów: $5000$
- środek okręgu: $(0,0)$
- promień okręgu: $R=300$

In [ ]:
dataset_b4 = generate_dataset_b((0,0),5000,300)


In [ ]:
pure_graham_algorithm(dataset_b4)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_b4, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.4")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_b4)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_b4, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.4")
plot.draw()

#### Zestaw danych B.5
- liczba punktów: $10000$
- środek okręgu: $(0,0)$
- promień okręgu: $R=300$

In [ ]:
dataset_b5 = generate_dataset_b((0,0),10000,300)


In [ ]:
pure_graham_algorithm(dataset_b5)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_b5, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.5")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_b5)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_b5, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.5")
plot.draw()

#### Zestaw danych B.6
- liczba punktów: $15000$
- środek okręgu: $(0,0)$
- promień okręgu: $R=300$

In [ ]:
dataset_b6 = generate_dataset_b((0,0),15000,300)


In [ ]:
pure_graham_algorithm(dataset_b6)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_b6, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.6")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_b6)

In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_b6, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.6")
plot.draw()

#### Zestaw danych B.7
- liczba punktów: $25000$
- środek okręgu: $(0,0)$
- promień okręgu: $R=300$

In [ ]:
dataset_b7 = generate_dataset_b((0,0),25000,300)


In [ ]:
pure_graham_algorithm(dataset_b7)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_b7, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.7")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_b7)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_b7, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.7")
plot.draw()

#### Zestaw danych B.8
- liczba punktów: $50000$
- środek okręgu: $(0,0)$
- promień okręgu: $R=300$

In [ ]:
dataset_b8 = generate_dataset_b((0,0),50000,300)


In [ ]:
pure_graham_algorithm(dataset_b8)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_b8, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych B.8")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_b8)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_b8, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych B.8")
plot.draw()

### Zestaw danych C

#### Zestaw danych C.1
- liczba punktów: $100$
- wierzchołki prostokąta: $(-10,-10), (10,-10),(10,10),(-10,10)$


In [ ]:
pure_graham_algorithm(default_dataset_c)


In [ ]:
lines, points, scene_container = graham_algorithm(default_dataset_c, True)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.1")
plot.draw()

In [ ]:
pure_jarvis_algorithm(default_dataset_c)


In [ ]:
lines, points, scene_container = jarvis_algorithm(default_dataset_c, True)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.1")
plot.draw()

#### Zestaw danych C.2
- liczba punktów: $1000$
- wierzchołki prostokąta: $(0,0),(30,0),(30,40),(0,40)$


In [ ]:
dataset_c2 = generate_dataset_c([(0,0),(30,0),(30,40),(0,40)],1000)


In [ ]:
pure_graham_algorithm(dataset_c2)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_c2, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.2")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_c2)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_c2, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.2")
plot.draw()

#### Zestaw danych C.3
- liczba punktów: $2500$
- wierzchołki prostokąta: $(0,0),(30,0),(30,40),(0,40)$


In [ ]:
dataset_c3 = generate_dataset_c([(0,0),(30,0),(30,40),(0,40)],2500)


In [ ]:
pure_graham_algorithm(dataset_c3)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_c3, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.3")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_c3)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_c3, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.3")
plot.draw()

#### Zestaw danych C.4
- liczba punktów: $5000$
- wierzchołki prostokąta: $(0,0),(30,0),(30,40),(0,40)$


In [ ]:
dataset_c4 = generate_dataset_c([(0,0),(30,0),(30,40),(0,40)],5000)


In [ ]:
pure_graham_algorithm(dataset_c4)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_c4, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.4")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_c4)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_c4, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.4")
plot.draw()

#### Zestaw danych C.5
- liczba punktów: $10000$
- wierzchołki prostokąta: $(0,0),(30,0),(30,40),(0,40)$


In [ ]:
dataset_c5 = generate_dataset_c([(0,0),(30,0),(30,40),(0,40)],10000)


In [ ]:
pure_graham_algorithm(dataset_c5)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_c5, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.5")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_c5)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_c5, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.5")
plot.draw()

#### Zestaw danych C.6
- liczba punktów: $15000$
- wierzchołki prostokąta: $(0,0),(30,0),(30,40),(0,40)$


In [ ]:
dataset_c6 = generate_dataset_c([(0,0),(30,0),(30,40),(0,40)],15000)


In [ ]:
pure_graham_algorithm(dataset_c6)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_c6, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.6")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_c6)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_c6, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.6")
plot.draw()

#### Zestaw danych C.7
- liczba punktów: $25000$
- wierzchołki prostokąta: $(0,0),(30,0),(30,40),(0,40)$


In [ ]:
dataset_c7 = generate_dataset_c([(0,0),(30,0),(30,40),(0,40)],25000)


In [ ]:
pure_graham_algorithm(dataset_c7)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_c7, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.7")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_c7)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_c7, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.7")
plot.draw()

#### Zestaw danych C.8
- liczba punktów: $50000$
- wierzchołki prostokąta: $(0,0),(30,0),(30,40),(0,40)$


In [ ]:
dataset_c8 = generate_dataset_c([(0,0),(30,0),(30,40),(0,40)],50000)


In [ ]:
pure_graham_algorithm(dataset_c8)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_c8, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych C.8")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_c8)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_c8, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych C.8")
plot.draw()

### Zestaw danych D

#### Zestaw danych D.1
- liczba punktów na bokach: $25$
- liczba punktów na przekątnych: $20$
- wierzchołki kwadratu: $(0,0),(10,0),(10,10),(0,10)$


In [ ]:
pure_graham_algorithm(default_dataset_d)

In [ ]:
lines, points, scene_container = graham_algorithm(default_dataset_d, True)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.1")
plot.draw()

In [ ]:
pure_jarvis_algorithm(default_dataset_d)

In [ ]:
lines, points, scene_container = jarvis_algorithm(default_dataset_d, True)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.1")
plot.draw()

#### Zestaw danych D.2
- liczba punktów na bokach: $1000$
- liczba punktów na przekątnych: $1000$
- wierzchołki kwadratu: $(0,0),(30,0),(30,30),(0,30)$


In [ ]:
dataset_d2 = generate_dataset_d(1000,1000,[(0,0),(30,0),(30,30),(0,30)])


In [ ]:
pure_graham_algorithm(dataset_d2)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_d2, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.2")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_d2)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_d2, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.2")
plot.draw()

#### Zestaw danych D.3
- liczba punktów na bokach: $2500$
- liczba punktów na przekątnych: $2500$
- wierzchołki kwadratu: $(0,0),(30,0),(30,30),(0,30)$


In [ ]:
dataset_d3 = generate_dataset_d(2500,2500,[(0,0),(30,0),(30,30),(0,30)])


In [ ]:
pure_graham_algorithm(dataset_d3)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_d3, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.3")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_d3)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_d3, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.3")
plot.draw()

#### Zestaw danych D.4
- liczba punktów na bokach: $5000$
- liczba punktów na przekątnych: $5000$
- wierzchołki kwadratu: $(0,0),(30,0),(30,30),(0,30)$


In [ ]:
dataset_d4 = generate_dataset_d(5000,5000,[(0,0),(30,0),(30,30),(0,30)])


In [ ]:
pure_graham_algorithm(dataset_d4)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_d4, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.4")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_d4)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_d4, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.4")
plot.draw()

#### Zestaw danych D.5
- liczba punktów na bokach: $10000$
- liczba punktów na przekątnych: $10000$
- wierzchołki kwadratu: $(0,0),(30,0),(30,30),(0,30)$


In [ ]:
dataset_d5 = generate_dataset_d(10000,10000,[(0,0),(30,0),(30,30),(0,30)])


In [ ]:
pure_graham_algorithm(dataset_d5)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_d5, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.5")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_d5)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_d5, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.5")
plot.draw()

#### Zestaw danych D.6
- liczba punktów na bokach: $15000$
- liczba punktów na przekątnych: $15000$
- wierzchołki kwadratu: $(0,0),(30,0),(30,30),(0,30)$


In [ ]:
dataset_d6 = generate_dataset_d(15000,15000,[(0,0),(30,0),(30,30),(0,30)])


In [ ]:
pure_graham_algorithm(dataset_d6)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_d6, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.6")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_d6)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_d6, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.6")
plot.draw()

#### Zestaw danych D.7
- liczba punktów na bokach: $25000$
- liczba punktów na przekątnych: $25000$
- wierzchołki kwadratu: $(0,0),(30,0),(30,30),(0,30)$


In [ ]:
dataset_d7 = generate_dataset_d(25000,25000,[(0,0),(30,0),(30,30),(0,30)])


In [ ]:
pure_graham_algorithm(dataset_d7)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_d7, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.7")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_d7)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_d7, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.7")
plot.draw()

#### Zestaw danych D.8
- liczba punktów na bokach: $50000$
- liczba punktów na przekątnych: $50000$
- wierzchołki kwadratu: $(0,0),(30,0),(30,30),(0,30)$


In [ ]:
dataset_d8 = generate_dataset_d(50000,50000,[(0,0),(30,0),(30,30),(0,30)])


In [ ]:
pure_graham_algorithm(dataset_d8)


In [ ]:
lines, points, scene_container = graham_algorithm(dataset_d8, False)
plot = Plot(scenes = scene_container, title="Algorytm Grahama - zbiór danych D.8")
plot.draw()

In [ ]:
pure_jarvis_algorithm(dataset_d8)


In [ ]:
lines, points, scene_container = jarvis_algorithm(dataset_d8, False)
plot = Plot(scenes = scene_container, title="Algorytm Jarvisa - zbiór danych D.8")
plot.draw()